In [3]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
from pyspark import SparkContext, SparkConf
import re
from operator import add
import wget

In [5]:
URL = 'http://www.almhuette-raith.at/apache-log/access.log'
logfile_name = wget.download(URL)

In [44]:
conf = SparkConf().setAppName("MyLocal")
sc = SparkContext(conf=conf)
lines = sc.textFile(logfile_name)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MyLocal, master=local[*]) created by __init__ at <ipython-input-20-fea3e09a4f1a>:2 

In [45]:
pattern = re.compile('(GET|POST) (.*?) ')

In [46]:
def process(x):
    val = re.search(pattern, x)
    return (val and val.groups()[1], 1)

In [47]:
lines.filter(lambda x: '404' in x).map(process).reduceByKey(add).sortBy(lambda x: x[1], ascending=False).take(10)


[('/templates/_system/css/general.css', 19882),
 ('/favicon.ico', 7757),
 ('/wp-login.php', 6287),
 ('/apache-log/favicon.ico', 2439),
 ('/apple-touch-icon.png', 1031),
 ('/apple-touch-icon-precomposed.png', 981),
 ('/apple-touch-icon-120x120-precomposed.png', 953),
 ('/apple-touch-icon-120x120.png', 952),
 ('/index.php?option=com_easyblog&view=dashboard&layout=write', 782),
 ('/wp-login.php?action=register', 706)]